TOOL USE PATTERN: Sports Edition

In [2]:
#simple fun that gets football match scores
import json

def get_match_score(team_name: str, league: str):
    """
    Get the latest match score for a football team
    
    team_name (str): The name of the team, e.g. "Manchester United", "Barcelona"
    league (str): The league name, e.g. "Premier League", "La Liga"
    """
    
    if team_name == "Manchester United":
        return json.dumps({
            "team": "Manchester United",
            "opponent": "Liverpool", 
            "score": "2-1",
            "status": "Final"
        })
    
    elif team_name == "Barcelona":
        return json.dumps({
            "team": "Barcelona",
            "opponent": "Real Madrid",
            "score": "3-2", 
            "status": "Final"
        })
    
    else:
        return json.dumps({
            "team": team_name,
            "score": "No recent match found"
        })

In [3]:
# Test it like a normal function
result = get_match_score(team_name="Manchester United", league="Premier League")
print(result)

{"team": "Manchester United", "opponent": "Liverpool", "score": "2-1", "status": "Final"}


In [4]:
#Teach AI about our Tool
import os
import re
from groq import Groq
from dotenv import load_dotenv

# Load API keys
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq()

# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_match_score",
    "description": "Get the latest match score for a football team. team_name (str): The name of the team, e.g. 'Manchester United', 'Barcelona'. league (str): The league name, e.g. 'Premier League', 'La Liga'",
    "parameters": {
        "properties": {
            "team_name": {
                "type": "str"
            },
            "league": {
                "type": "str"
            }
        }
    }
}
</tools>
"""

In [5]:
import sys
sys.path.append("../src") 

In [6]:
#Ask AI a Question
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]
agent_chat_history = []
user_msg = {
    "role": "user",
    "content": "What's the latest score for Manchester United in the Premier League?"
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<tool_call>
{"name": "get_match_score","arguments": {"team_name": "Manchester United", "league": "Premier League"}}
</tool_call>


In [7]:
#Parsing AI response
def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)
    
    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output"

In [8]:
parsed_output = parse_tool_call_str(output)
print(parsed_output)

{'name': 'get_match_score', 'arguments': {'team_name': 'Manchester United', 'league': 'Premier League'}}


In [9]:
#Running the tool
result = get_match_score(**parsed_output["arguments"])
print(result)

{"team": "Manchester United", "opponent": "Liverpool", "score": "2-1", "status": "Final"}


In [10]:
agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {result}"
})

In [11]:
GROQ_CLIENT.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

'According to the latest observation, the score for Manchester United in their Premier League match against Liverpool is 2-1 in favor of Manchester United, and the match has ended with a final status.'

In [24]:
import json
import requests
from dotenv import load_dotenv
from groq import Groq
from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.tool_pattern.tool_agent import ToolAgent

load_dotenv()
GROQ_CLIENT = Groq()
MODEL = "llama-3.3-70b-versatile"


@tool
def get_live_match_score(team_name: str):
    """Get latest match score using free public API"""
    
    url = f"https://www.thesportsdb.com/api/v1/json/3/searchteams.php?t={team_name}"

    try:
        response = requests.get(url)
        data = response.json()

        if data['teams']:
            team_info = data['teams'][0]
            team_id = team_info['idTeam']

            matches_url = f"https://www.thesportsdb.com/api/v1/json/3/eventslast.php?id={team_id}"
            matches_response = requests.get(matches_url)
            matches_data = matches_response.json()

            if matches_data['results']:
                latest = matches_data['results'][0]
                return {
                    'home_team': latest['strHomeTeam'],
                    'away_team': latest['strAwayTeam'],
                    'home_score': latest['intHomeScore'],
                    'away_score': latest['intAwayScore'],
                    'date': latest['dateEvent'],
                    'status': latest['strStatus']
                }

        return {'error': f'No data found for {team_name}'}

    except Exception as e:
        return {'error': str(e)}

agent = ToolAgent(
    tools=[get_live_match_score],
    model=MODEL  
)


def run_agent(query: str):
    result = agent.run(query)
    print(result)


In [28]:
run_agent("What was Barcelona's last game score?")
run_agent("Tell me about Liverpool's most recent match")
run_agent("Did Real Madrid win their last game?")


Using Tool: get_live_match_score

Tool call dict: 
{'name': 'get_live_match_score', 'arguments': {'team_name': 'Barcelona'}, 'id': 1}

Tool result: 
{'home_team': 'Barcelona', 'away_team': 'Elche', 'home_score': '3', 'away_score': '1', 'date': '2025-11-02', 'status': 'Match Finished'}
Barcelona's last game score was 3-1, where they were the home team and played against Elche. The game was played on November 2, 2025, and has been marked as "Match Finished".

Using Tool: get_live_match_score

Tool call dict: 
{'name': 'get_live_match_score', 'arguments': {'team_name': 'Liverpool'}, 'id': 1}

Tool result: 
{'home_team': 'Liverpool', 'away_team': 'Real Madrid', 'home_score': '1', 'away_score': '0', 'date': '2025-11-04', 'status': 'Match Finished'}
According to the data, Liverpool's most recent match was against Real Madrid on November 4, 2025. The match has finished, and the result is:

Liverpool (Home) 1 - 0 Real Madrid (Away)

So, Liverpool won the match with a score of 1-0.

Using Tool